### <center> 5. Строковые данные: основные типы

В *Postgres* есть три основных типа данных для работы со строками: `character`, `character varying` и `text`.

Кратко представим каждый из них.

**character** - строка фиксированной длины, дополненная пробелами.
Длина строки такого типа всегда одинакова и задаётся в скобках.

Основной паттерн **использования** такого типа — универсальные справочники буквенных кодов

**CHARACTER VARYING**

**character varying** - Строка ограниченной переменной длины.

Например, в столбце типа *character varying(5)* нельзя будет хранить строку большей длины, но могут быть любые строки с меньшей длиной.

Этот тип данных повсеместно используется для хранения данных, поскольку позволяет ограничить ввод, сохраняя при этом возможность иметь строки произвольной длины.

**TEXT**

**text** - Cтрока неограниченной длины.

Самый удобный тип для пользователя, но самый тяжеловесный для администратора баз данных, так как в строку можно записать любой текст.

### <center> 6. Функции и операторы для работы со строками

##### <center> ОПЕРАТОРЫ

**СОЕДИНЕНИЕ СТРОК**

`||` - Оператор конкатенации (соединения) строк. Позволяет объединить 2 и более строк.

Конструкции с оператором соединения строк записываются следующим образом:

`строка1 || строка2 || ... || строкаN`

**Важно!** Результатом соединения любых типов строковых данных будет тип *text*.



Напишем запрос, который позволит подготовить простые select-запросы для всех таблиц из схемы.

```sql
select 'select * from '||t.table_schema||'.'||t.table_name||';' query
from information_schema.tables t
where table_schema = 'shipping'
```

В результате должно получиться пять SQL-запросов, по одному к каждой таблице из схемы `shipping`. 

Как мы видим, соединять можно и рукописный текст, и значения столбцов в любом произвольном порядке.

>**Важно!** Если вы соединяете любую строку и *NULL*, то результатом будет *NULL*. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует *NULL*, используйте оператор [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull).

**Задание 6.1**

Составим текстовый шаблон сообщения о доставке по конкретному водителю для наших клиентов. Напишите SQL-запрос, который выведет следующее сообщение для каждого водителя по форме:

Ваш заказ доставит водитель #Имя Фамилия# (first_name, last_name таблица driver). Его контактный номер: #Номер# (phone таблица driver)

Где #Имя Фамилия# и #Номер# взяты из справочника водителей. Если номер не указан, то выведите прочерк (-). Для номеров рекомендуем использовать COALESCE. Пример из таблицы для наглядности:

Ваш заказ доставит водитель Adel Al-Alawi. Его контактный номер: (901) 947-4433

Столбец к выдаче — msg (текст сообщения).

```sql
select
    'Ваш заказ доставит водитель '||first_name||' '||last_name||'.'||' '||'Его контактный номер: '|| COALESCE(phone, '-') as msg
from
    sql.driver
```

##### <center> ФУНКЦИИ

**UPPER() И LOWER()**

`upper(your_text)` и `lower(your_text)` -Функции, которые переводят каждый символ вашего текста в верхний и нижний регистр соответственно.

```sql
select upper('Abc') s1 ,lower('xYz') s2
```

Чаще всего эти функции используются для унификации и стандартизации, особенно они актуальны для данных, введённых вручную.

Например, названия города в анкете можно написать разными способами, но символьный состав останется одним и тем же (Москва, москва, МОСКВА).

**Задание 6.2**

Cоставим справочник названий клиентов (таблица customer), у которых более десяти доставок (таблица shipment). Данные сохраним в нижнем регистре, чтобы передавать их в другие системы (например, для обзвона), которые не чувствительны к регистру. Напишите запрос, который выводит все id названий клиентов (cust_id), у которых более десяти доставок, в нижнем регистре. Отсортируйте результат по cust_id в порядке возрастания. Столбцы в выдаче: cust_id (id клиента) и cust_name (название клиента в нижнем регистре).

```sql
SELECT
    c.cust_id,
    LOWER(c.cust_name) as cust_name
FROM
    sql.customer as c
    JOIN sql.shipment as s ON c.cust_id = s.cust_id
GROUP BY
    c.cust_id,
    c.cust_name -- В PostgreSQL и SQL Server необходимо включать неагрегированные столбцы в GROUP BY
HAVING
    COUNT(s.ship_id) > 10 --Подсчитываем количество доставок (или просто COUNT(*))
ORDER BY
    c.cust_id ASC
```

**REPLACE()**

`replace()` - Функция, с помощью которой можно заменять символы в строках.

`replace(string text (где?), from text (что?), to text (на что?))`

```sql
select replace('малако','а','о')
```
Эта запись означает, что в исходной строке *string* мы заменяем все вхождения строки `from` на строку `to`.

```sql
select replace('малако','а','о')
```

Результат выполнения такого запроса будет молоко, т. е. все буквы «а» в строке «малако» были заменены на «о».

С таким же успехом можно заменять строку, состоящую из нескольких символов.

```sql
SELECT replace('машина', 'шина', 'трас')
```
В строке 'машина' мы заменили строку 'шина' на строку 'трас'.

Результат функции `replace()` — строка, а значит, к ней тоже можно применять все известные нам функции работы со строками.

>Если вы хотите удалить из строки какие-то символы, то третьим параметром (to) передайте пустую строку ''(одинарные кавычки без символа внутри).

Например, сделаем из строки "Hello, world!" строку "Hello!".

```sql
select replace('Hello, world!',', world','')
```

**Задание 6.3**

Составим справочник utm-меток, для того чтобы передавать город (city_name тaблица city) и штат (state, таблица city) прямо в адресной строке. (Если вы не знаете, что такое utm-метка, почитайте статью на Вики. К программе курса это не относится, но знать полезно.) Напишите SQL-запрос, который выведет список сочетаний из справочника следующего вида: название_штата__название_города, где названия штата и города взяты из справочника городов и переведены в нижний регистр. Столбец к выдаче — utm (форматированный штат-город). Отсортируйте полученный справочник по алфавиту. Обратите внимание! Все пробелы в названиях городов и штатов замените символом '_' (одно нижнее подчёркивание), а для разделения названий города и штата используйте '__' (два последовательных нижних подчёркивания). Пример из таблицы для наглядности: new_jersey__union_city

```sql
SELECT
    replace(lower(state), ' ', '_') || '__' || replace(lower(city), ' ', '_') as utm
FROM
    sql.city
ORDER BY
    utm ASC
```

**LEFT() И RIGHT()**

`left(string,n)` и `right(string,n)` - Функции, которые оставляют n левых или правых символов от строки, поданной на вход.

Разобьём строку 'Один два три' на слова, используя эти функции.

```sql
with t as
(
    SELECT 'Один два три'::text sample_string
)
SELECT
    left(t.sample_string, 4) one, /*берём 4 левых символа строки*/
    right(left(t.sample_string, 8), 3) two, /*берём 8 левых символов строки, потом 3 правых от результата*/
    right(t.sample_string, 3) three /*берём 3 правых символа от строки*/
FROM t
```

```sql
select left('0123456789', - 2), right('0123456789', - 2)
```
```sql
Результат: 01234567 и 23456789 (в первом случае — восемь символов с «отрезанными» 89 и во втором случае — восемь символов с «отрезанными» 01)

**Задание 6.4**

Представим, что к вам пришёл разработчик, который хочет сократить поле state в таблице city до четырёх символов, и попросил проверить, останeтся ли значения в нём уникальными. Чтобы ответить на этот вопрос, напишите SQL-запрос, который выведет первые четыре символа названия штата (state, тaблица city) и количество уникальных названий штатов, которому они соответствуют. Оставьте только те, которые относятся к двум и более штатам. Добавьте сортировку по первому столбцу. Столбцы в выдаче: code (четыре первых символа в названии штата), qty (количество уникальных названий штата, начинающихся с этих символов).

```sql
SELECT
    left(state, 4) as code,
    COUNT(DISTINCT state)
FROM sql.city
GROUP BY 
    code
HAVING
    COUNT(DISTINCT state) >= 2
ORDER BY
    code
```

**FORMAT()**

`format()` - используется для составления форматированного текста с подстановками. 

**Синтаксис:**

`format(formatstr text [, argument1 text,argument2 text...])`

где `formatstr` — это шаблон, который мы передаём. Это обычная строка, в которой указаны места для подстановки аргумента.

```sql
SELECT format('Hello, %s!', d.first_name) from shipping.driver as d
```
Комбинация символов `%s` обозначает, что вместо них будет подставлен один из аргументов, причём в том же порядке, что и в исходном столбце.


Напишем запрос, который описывает содержимое каждой строки в таблице в виде текста.

```sql
SELECT format('driver_id = %s, first_name = %s, last_name = %s, address = %s, zip_code = %s, phone = %s, city_id = %s', driver_id, first_name, last_name, address, zip_code, phone, city_id) from shipping_driver as d
```

Мы перечислили в строке семь пропусков (плэйсхолдеров, или мест для подстановки, — %s), передали семь параметров (все столбцы таблицы) и получили шаблон, заполненный значениями для каждой строки.

Если в вашем шаблоне присутствует одинарная кавычка, то для удобства можно вместо одинарных кавычек использовать $$ (два знака доллара):

```sql
select $$ some_string with quotes ' $$
```

**Задание 6.5**

Давайте подготовим географическую сводку для каждого города. Напишите SQL-запрос, который выведет описание региона в следующем формате:


[city_name] is located in [state]. There's [population] people living there. Its area is [area]

Обратите внимание, точку в конце ставить не нужно. Отсортируйте по названию города в алфавитном порядке. Столбец к выдаче — str (сводка). Пример:


Abilene is located in Texas. There's 115930 people living there. Its area is 105.10

```sql
SELECT
    FORMAT('%s is located in %s. There''s %s people living there. Its area is %s', 
           city_name, state, population, area) AS str
FROM
    sql.city
ORDER BY
    city_name ASC;
```